## import test set from corpus

## import the training set from corpus

In [73]:
# select data for training
file_train = easygui.fileopenbox(default = "D:/")
sents_train = conllu.parse(open(file, encoding = "utf-8-sig").read())

# arrange the data in format [("text", {"heads": heads, "deps": deps }), ...]
train_data = []
for sent in sents_train:
    # extract head index
    heads = [token["head"] for token in sent]
    # extract dependency
    deps = [token["deprel"] for token in sent]
    # combine deps and heads
    dep_dict = {"heads": heads,"deps": deps}
    # insert raw text
    train_data.append((sent.metadata["text"], dep_dict))

In [95]:
train_data[:10]

[('which united airlines flight flies across this continent and leaves from boston goes to dallas and makes a stopover in philadelphia',
  {'heads': [2,
    4,
    2,
    5,
    0,
    8,
    8,
    5,
    10,
    5,
    12,
    10,
    5,
    15,
    13,
    17,
    5,
    19,
    17,
    21,
    17],
   'deps': ['det',
    'compound',
    'flat',
    'nsubj',
    'root',
    'case',
    'det',
    'obl',
    'cc',
    'conj',
    'case',
    'obl',
    'conj',
    'case',
    'obl',
    'cc',
    'conj',
    'det',
    'obl',
    'case',
    'obl']}),
 ('which nonstop flight from atlanta to oakland leaves in the afternoon and arrives at 5 pm',
  {'heads': [3, 3, 8, 5, 3, 7, 3, 0, 11, 11, 8, 13, 8, 16, 16, 13],
   'deps': ['det',
    'compound',
    'nsubj',
    'case',
    'nmod',
    'case',
    'nmod',
    'root',
    'case',
    'det',
    'obl:tmod',
    'cc',
    'conj',
    'case',
    'nummod',
    'obl:tmod']}),
 ('i need a sunday flight from tampa to charlotte',
  {'heads': 

## train the parser

In [3]:
# choose language
language = easygui.choicebox(msg = None, title = "Language", choices = ("English","Spanish"))
if language == "English":
    model_name = "en"
else:
    model_name = "es"

In [94]:
import spacy
nlp = spacy.blank(model_name)

# updata the default parser

nlp.create_pipe(["parser","tok2vec"]) # create parser and load tok2vec
nlp.add_pipe(["parser","tok2vec"])

# add labels

# all the labels from annotated corpus
labels = ["acl:relcl","advcl","advcl:relcl","advmod","amod","appos","aux","aux:pass","case","cc","cc:preconj","ccomp","compound","compound:prt",
"conj","cop","csubj","csubj:outer","csubj:pass","dep","det","det:predet","discourse","dislocated","expl","fixed","flat","flat:foreign",
"goeswith","iobj","list","mark","nmod","nmod:npmod","nmod:poss","nmod:tmod","nsubj","nsubj:outer","nsubj:pass","nummod","obj","obl","obl:npmod","obl:tmod","orphan","parataxis","punct","reparandum","root","vocative","xcomp"]

# add label to the default parser
parser = nlp.get_pipe("parser")
for label in labels:
    if label not in parser.labels:
        parser.add_label(label)
        
# create examples
from spacy.training import Example
examples = []
for text, dep_annotations in train_data:
    doc = nlp.make_doc(text)
    # convert the train_data into example
    examples.append(Example.from_dict(doc, dep_annotations))

# training

import random

# get number of iterations
training_iterations = easygui.integerbox(msg="How many times of iteration:", lowerbound = 1)

# initialize the model
optimizer = nlp.begin_training()
for i in range(training_iterations):
    random.shuffle(example)
    # update the model
    nlp.update(example, sgd = optimizer) 

print("Training finished.")

ValueError: [E190] Token head out of range in `Doc.from_array()` for token index '19' with value '2' (equivalent to relative head index: '2'). The head indices should be relative to the current token index rather than absolute indices in the array.